# Automated Testing with AI

In [ ]:
# %pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Selenium-based login test (valid and invalid credentials)
# Replace URL, selectors and credentials for your target app.

def create_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
        opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)
    driver.set_window_size(1200, 800)
    return driver

def login_and_check(url, username, password, selectors, success_selector, timeout=8, headless=True):
    """
    Attempts login and returns True if success_selector is found after login, else False.
    selectors: dict with keys 'username', 'password', 'submit' (each a (By, value) tuple)
    success_selector: (By, value) tuple expected to appear on successful login (e.g. logout button or dashboard)
    """
    driver = create_driver(headless=headless)
    try:
        driver.get(url)
        # fill username
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located(selectors['username'])).send_keys(username)
        # fill password
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located(selectors['password'])).send_keys(password)
        # click submit
        submit_el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(selectors['submit']))
        submit_el.click()
        # wait a short while for page to update
        try:
            WebDriverWait(driver, timeout).until(EC.presence_of_element_located(success_selector))
            return True
        except:
            return False
    finally:
        time.sleep(0.5)
        driver.quit()

# Example configuration - MODIFY for your application
login_url = "https://academy.powerlearnprojectafrica.org/login"  # change to real login page
selectors = {
    # common patterns: (By.ID, "username"), (By.NAME, "username"), (By.CSS_SELECTOR, "input#user")
    "username": (By.ID, "username"),
    "password": (By.ID, "password"),
    "submit":   (By.CSS_SELECTOR, "button[type='submit']"),
}
# Element that indicates successful login (e.g., logout button or user profile)
success_selector = (By.ID, "logout")  # change to a selector visible after successful login

# Test credentials (change to valid/invalid credentials for your app)
valid_creds = ("valid_user", "valid_pass")
invalid_creds = ("invalid_user", "wrong_pass")

# Run tests
valid_result = login_and_check(login_url, valid_creds[0], valid_creds[1], selectors, success_selector, headless=True)
invalid_result = login_and_check(login_url, invalid_creds[0], invalid_creds[1], selectors, success_selector, headless=True)

print(f"Valid credentials test: {'PASS' if valid_result else 'FAIL'}")
print(f"Invalid credentials test: {'FAIL' if not invalid_result else 'UNEXPECTED PASS'}")